<a href="https://colab.research.google.com/github/HAUCommunityFridge/James_Fareshare_Data/blob/main/scripts/datacleaning.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [5]:
# Imports
import pandas as pd
import os
import re
from openpyxl import load_workbook
from datetime import datetime

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
#@title **Step 1: Clone the GitHub Repository**
# This step clones the required GitHub repository into the Colab environment.

# Define the path where the repository will be cloned
repo_path = '/content/James_Fareshare_Data'

# Check if the repository already exists
if not os.path.exists(repo_path):
    # Clone the repository if it doesn't exist
    !git clone https://github.com/HAUCommunityFridge/James_Fareshare_Data
else:
    # If the repository exists, pull the latest changes
    os.chdir(repo_path)
    !git pull

# Verify cloned repository
print("Repository contents:")
!ls /content/James_Fareshare_Data

Already up to date.
Repository contents:
data  docs  James_Fareshare_Data.Rproj	scripts


In [ ]:
#@title **Step 2: Inital loading**

In [ ]:
# Define the file path to the Excel file
file_path = "/content/James_Fareshare_Data/data/Fareshare records from April to July plus.xlsx"

# Load the workbook and get the sheet names
workbook = load_workbook(filename=file_path, read_only=True)
sheet_names = workbook.sheetnames

# Specify the sheets to ignore based on their indices
# These sheets may contain metadata or irrelevant information
sheets_to_ignore = [0, 1, 7, 10, 12, 13, 14, 15]

# Exclude the specified sheets from the list of sheets to load
sheets_to_load = [sheet for idx, sheet in enumerate(sheet_names) if idx not in sheets_to_ignore]

In [ ]:
#@title **Step 3: Inital data cleaning and merging**

In [ ]:
# Define the standard column names to be used across all sheets
column_names = ["Date", "Category", "Product.brand..name.and.type", "Barcode", "Quantity", "Unit",
                "Unit.weight", "Total.weight", "Unit.price", "Total.price", "SheetName"]

# Helper function to standardize column names by removing non-alphabetical characters and converting to lowercase
def standardize_column_name(name):
    if isinstance(name, str):
        return re.sub(r'[^a-z]', '', name.lower())
    return name

# Function to clean the 'Total.price' column by removing the 'GBP' suffix and converting to a float
def clean_total_price(price):
    if isinstance(price, str) and 'GBP' in price:
        return float(price.replace(' GBP', '').strip())
    return price

# Function to load and clean each sheet
def clean_sheet(sheet_name):
    # Load the sheet into a DataFrame
    data = pd.read_excel(file_path, sheet_name=sheet_name)

    # Print the first few rows of the original sheet for inspection
    print(f"Original data from sheet {sheet_name}:\n", data.head())

    # Standardize the column names
    data.columns = [standardize_column_name(col) for col in data.columns]

    # Print actual column names for debugging
    print(f"Sheet: {sheet_name}, Columns: {data.columns.tolist()}")

    # Limit columns to the length of column_names
    data = data.iloc[:, :len(column_names)]

    # Print actual number of columns detected for debugging
    print(f"Sheet: {sheet_name}, Columns Detected: {len(data.columns)}")

    # Adjust the number of columns if necessary
    if len(data.columns) > len(column_names):
        data = data.iloc[:, :len(column_names)]
    elif len(data.columns) < len(column_names):
        data = data.reindex(columns=column_names[:len(data.columns)])

    # Rename columns to the standard column names
    data.columns = column_names[:len(data.columns)]

    # Ensure only the required columns are kept
    data = data[column_names[:len(data.columns)]]

    # Remove rows with all NaN values
    data = data.dropna(how='all')

    # Process the 'Date' column: convert to datetime and handle missing dates
    if "Date" in data.columns:
        data['Date'] = pd.to_datetime(data['Date'], errors='coerce', dayfirst=True).dt.date
        if sheet_name == "Malinslee":
            data['Date'] = pd.NA  # Set date to NaN for Malinslee sheet if needed
        else:
            data['Date'] = data['Date'].ffill()  # Forward fill for other sheets

    # Clean the 'Total.price' column
    if 'Total.price' in data.columns:
        data['Total.price'] = data['Total.price'].apply(clean_total_price)
        data['Total.price'] = pd.to_numeric(data['Total.price'], errors='coerce')

    # Remove rows containing 'GBP' for now
    data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)

    # Drop columns containing only NA values
    data = data.dropna(axis=1, how='all')

    # Add a column for the sheet name to track the origin of the data
    data['SheetName'] = sheet_name

    # Check for missing values in the cleaned data
    missing_values = data.isna().sum()
    print(f"Missing values in sheet {sheet_name}:\n{missing_values}")

    # Print the first few rows of the cleaned data
    print(f"Cleaned data from sheet {sheet_name}:\n", data.head())

    return data

# Load and clean all relevant sheets, storing each cleaned DataFrame in a list
data_list = [clean_sheet(sheet) for sheet in sheets_to_load]

# Remove NULL elements from the list, if any
data_list = [data for data in data_list if data is not None]

# Combine all cleaned data into a single DataFrame
if data_list:
    consolidated_data = pd.concat(data_list, ignore_index=True)

    # Remove rows where 'Category' is NaN, as they may not be useful
    consolidated_data = consolidated_data[consolidated_data['Category'].notna()]

    # Remove duplicate rows
    consolidated_data = consolidated_data.drop_duplicates()

    # Check for missing values in the consolidated data
    missing_values_consolidated = consolidated_data.isna().sum()
    print(f"Missing values in consolidated data:\n{missing_values_consolidated}")

    # Save the cleaned data to a new Excel file
    output_file_path = "/content/Cleaned_Fareshare_data_final.xlsx"
    consolidated_data.to_excel(output_file_path, index=False)

    print(f"Cleaned data saved to {output_file_path}")

    # Display the first few rows of the cleaned DataFrame
    print(consolidated_data.head())

    # Display the structure of the cleaned DataFrame
    print(consolidated_data.info())
else:
    print("No valid data found to consolidate.")

Original data from sheet Hadley:
     Received          Category  \
0 2024-05-01  Fresh vegetables   
1 2024-05-01               NaN   
2 2024-05-01  Fresh vegetables   
3 2024-05-01               NaN   
4 2024-05-01  Fresh vegetables   

                        Product brand, name and type       Barcode  Quantity  \
0             Tesco Mixed Raisins 60G, Fruit and Veg  3.424605e+06       8.0   
1                                                NaN           NaN       NaN   
2  Tesco Sour Cream Chive Flavoured Pulse & Nut M...  5.059697e+12       7.0   
3                                                NaN           NaN       NaN   
4        Tesco Mixed Peppers 420G (C), Fruit and Veg  3.274880e+06       1.0   

  Unit  Unit weight  Total weight Unit price  Total price  Expiry date  \
0   Kg         0.06          0.51       1.15         9.20          NaN   
1  NaN          NaN           NaN       GBP           NaN          NaN   
2   Kg         0.03          0.18       0.75         5.95 

<ipython-input-67-7ec9b272f6d2>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)


Original data from sheet Market Drayton (July):
               Received          Category  \
0  2024-07-01 00:00:00  Fresh vegetables   
1  2024-07-01 00:00:00               NaN   
2  2024-07-01 00:00:00  Fresh vegetables   
3                  NaN               NaN   
4                  NaN  Fresh vegetables   

                        Product brand, name and type       Barcode  Quantity  \
0                Tesco Courgettes (C), Fruit and Veg  5.057754e+12       3.0   
1                                                NaN           NaN       NaN   
2  Tesco Flat Peach Mineral 3 Pack (C), Fruit and...  3.259412e+06       1.0   
3                                                NaN           NaN       NaN   
4      Tesco Iceberg Lettuce Each (C), Fruit and Veg  3.043868e+06       1.0   

  Unit  Unit weight  Total weight Unit price Total price  ...  \
0   Kg         0.67          2.00        1.5   4.50 GBP   ...   
1  NaN          NaN           NaN       GBP          NaN  ...   
2   Kg    

<ipython-input-67-7ec9b272f6d2>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)


Original data from sheet Market Drayton (June):
     Received                              Category  \
0 2024-06-02  Chilled products with dairy and eggs   
1 2024-06-02                                   NaN   
2        NaT  Chilled products with dairy and eggs   
3        NaT                                   NaN   
4        NaT  Chilled products with dairy and eggs   

                        Product brand, name and type       Barcode  Quantity  \
0             Tesco Chicken Drumsticks 600G, Chilled  5.054269e+12       1.0   
1                                                NaN           NaN       NaN   
2    Tesco British Whole Milk 568Ml, 1 Pint, Chilled  5.031021e+12       9.0   
3                                                NaN           NaN       NaN   
4  Tesco Organic British Whole Milk 2.272L/4 Pint...  5.050180e+12       6.0   

  Unit  Unit weight  Total weight Unit price Total price  ...  \
0   Kg         0.70          0.70        2.3   2.70 GBP   ...   
1  NaN         

...

Original data from sheet Market Drayton (April):
     Received          Category  \
0 2024-04-22  Fresh vegetables   
1        NaT               NaN   
2        NaT  Fresh vegetables   
3        NaT               NaN   
4        NaT  Fresh vegetables   

                        Product brand, name and type     Barcode  Quantity  \
0     Jaffa Orange Minimum 4 Pack (C), Fruit and Veg  10004241.0       3.0   
1                                                NaN         NaN       NaN   
2                    Lemons 4Pack (C), Fruit and Veg  10098752.0       9.0   
3                                                NaN         NaN       NaN   
4  Tesco Granny Smith Apple 6 Pack (C), Fruit and...   3249499.0       3.0   

  Unit  Unit weight  Total weight Unit price  Total price  ...  Weight  \
0   Kg         0.75          2.24          2         6.90  ...     NaN   
1  NaN          NaN           NaN       GBP           NaN  ...     NaN   
2   Kg         0.50          4.49        1.4        12

<ipython-input-67-7ec9b272f6d2>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)
<ipython-input-67-7ec9b272f6d2>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)


Original data from sheet Malinslee:
     Received                              Category  \
0 2024-05-02  Chilled products with dairy and eggs   
1 2024-05-02                                   NaN   
2 2024-05-02  Chilled products with dairy and eggs   
3 2024-05-02                                   NaN   
4 2024-05-02                      Fresh vegetables   

                        Product brand, name and type       Barcode  Quantity  \
0  Tesco British Chicken Breast Mini Fillets 400G...  5.057754e+12       1.0   
1                                                NaN           NaN       NaN   
2        Tesco Chicken Breast Portions 580G, Chilled  5.057754e+12       3.0   
3                                                NaN           NaN       NaN   
4    Tesco Sweet & Crunchy Salad 250G, Fruit and Veg  3.336922e+06       2.0   

  Unit  Unit weight  Total weight Unit price  Total price  ...  \
0   Kg         0.44          0.44       3.75         3.75  ...   
1  NaN          NaN      

<ipython-input-67-7ec9b272f6d2>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)
<ipython-input-67-7ec9b272f6d2>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if isinstance(x, str) and 'GBP' in x else x)


Cleaned data saved to /content/Cleaned_Fareshare_data_final.xlsx
         Date          Category  \
0  2024-05-01  Fresh vegetables   
2  2024-05-01  Fresh vegetables   
4  2024-05-01  Fresh vegetables   
6  2024-05-01  Fresh vegetables   
8  2024-05-01  Fresh vegetables   

                        Product.brand..name.and.type       Barcode  Quantity  \
0             Tesco Mixed Raisins 60G, Fruit and Veg  3.424605e+06       8.0   
2  Tesco Sour Cream Chive Flavoured Pulse & Nut M...  5.059697e+12       7.0   
4        Tesco Mixed Peppers 420G (C), Fruit and Veg  3.274880e+06       1.0   
6       Tesco Pink Lady Apple Pot 80G, Fruit and Veg  5.057753e+12       2.0   
8  Tesco Nut Bites Mix Baklava Flavour 25G, Fruit...  5.059698e+12      10.0   

  Unit  Unit.weight  Total.weight Unit.price  Total.price SheetName  
0   Kg         0.06          0.51       1.15         9.20    Hadley  
2   Kg         0.03          0.18       0.75         5.95    Hadley  
4   Kg         0.57          0.57

In [ ]:
#@title **Step 4: Waitrose data cleaning and merging**

In [ ]:
# Load the Waitrose sheet data to inspect it
waitrose_data = pd.read_excel(file_path, sheet_name="Waitrose")

# Display the first few rows of the raw data to understand its structure
print("Raw Waitrose Data:")
print(waitrose_data.head(50))

Raw Waitrose Data:
     Received                              Category  \
0  2024-05-29  Chilled products with dairy and eggs   
1         NaT                          Chilled fish   
2         NaT                          Chilled meat   
3         NaT              Bread and bread products   
4         NaT                      Fresh vegetables   
5         NaT                           Fresh fruit   
6         NaT                           Fresh fruit   
7         NaT                           Fresh fruit   
8         NaT                                   NaN   
9  2024-06-08  Chilled products with dairy and eggs   
10        NaT                              Category   
11        NaT  Chilled products with dairy and eggs   
12        NaT                          Chilled fish   
13        NaT                          Chilled meat   
14        NaT              Bread and bread products   
15        NaT                      Fresh vegetables   
16        NaT                           Fresh 

In [ ]:
# Helper function to standardise column names
def standardize_column_name(name):
    if isinstance(name, str):
        return re.sub(r'[^a-z]', '', name.lower())
    return name

# Function to clean the Waitrose sheet
def clean_waitrose_sheet(data):
    # Standardise the column names
    data.columns = [standardize_column_name(col) for col in data.columns]

    # Convert empty strings to NaN
    data.replace("", pd.NA, inplace=True)

    # Fill missing Date values with the last valid observation forward
    data['received'] = data['received'].replace('nan', pd.NA)  # Treat 'nan' as NA
    data['received'].fillna(method='ffill', inplace=True)  # Forward fill the date

    # Ensure the 'received' column is treated as a string before applying string operations
    data['received'] = data['received'].astype(str)

    # Remove duplicate headers
    data = data[~data['received'].str.contains('Category', na=False)]

    # Drop completely empty rows
    data.dropna(how='all', inplace=True)

    # Replace remaining NaN values with "N/A"
    data.fillna(value="N/A", inplace=True)

    # Standardise column names to match the other datasets
    data.rename(columns={
        'received': 'Date',
        'category': 'Category',
        'productbrandnameandtype': 'Product.brand..name.and.type',
        'barcode': 'Barcode',
        'quantity': 'Quantity',
        'unit': 'Unit',
        'kgperunit': 'Unit.weight',
        'totalweight': 'Total.weight',
        'unitprice': 'Unit.price',
        'totalprice': 'Total.price',
        'commentdonated': 'Comment/Donated'
    }, inplace=True)

    # Format Date to remove time
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce').dt.date

    # Add a column for the sheet name
    data['SheetName'] = 'Waitrose'

    # Rearrange columns to match the desired output
    column_order = ["Date", "Category", "Product.brand..name.and.type", "Barcode", "Quantity",
                    "Unit", "Unit.weight", "Total.weight", "Unit.price", "Total.price", "SheetName"]
    data = data[column_order]

    # Print the cleaned data for debugging
    print(f"Cleaned Waitrose Data:\n{data.head()}")

    return data

# Load the Waitrose sheet
file_path = "/content/James_Fareshare_Data/data/Fareshare records from April to July plus.xlsx"
waitrose_data = pd.read_excel(file_path, sheet_name="Waitrose")

# Apply the custom cleaning function to the Waitrose data
cleaned_waitrose_data = clean_waitrose_sheet(waitrose_data)

# Save the cleaned Waitrose data to an Excel file
waitrose_output_file_path = "/content/Cleaned_Waitrose_Data.xlsx"
cleaned_waitrose_data.to_excel(waitrose_output_file_path, index=False)

print(f"Cleaned Waitrose data saved to {waitrose_output_file_path}")

Cleaned Waitrose Data:
         Date                              Category  \
0  2024-05-29  Chilled products with dairy and eggs   
1  2024-05-29                          Chilled fish   
2  2024-05-29                          Chilled meat   
3  2024-05-29              Bread and bread products   
4  2024-05-29                      Fresh vegetables   

  Product.brand..name.and.type Barcode Quantity   Unit Unit.weight  \
0                          N/A     N/A      2.5  crate         6.5   
1                          N/A     N/A      0.5  crate         6.5   
2                          N/A     N/A        1  crate         6.5   
3                          N/A     N/A      1.5  crate         6.5   
4     Lettuce and spring onion     N/A        3  crate         6.5   

  Total.weight Unit.price Total.price SheetName  
0        16.25        N/A         N/A  Waitrose  
1         3.25        N/A         N/A  Waitrose  
2          6.5        N/A         N/A  Waitrose  
3         9.75        N/A

<ipython-input-70-52f71b27336d>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['received'].fillna(method='ffill', inplace=True)  # Forward fill the date


In [ ]:
# Load the previously cleaned data
cleaned_data = pd.read_excel("/content/Cleaned_Fareshare_data_final.xlsx")

# Append the cleaned Waitrose data
final_combined_data = pd.concat([cleaned_data, cleaned_waitrose_data], ignore_index=True)

# Save the final combined dataset
final_output_file_path = "/content/Combined_Fareshare_With_Waitrose_Data.xlsx"
final_combined_data.to_excel(final_output_file_path, index=False)

print(f"Final combined data saved to {final_output_file_path}")

# Load the final combined dataset to verify
final_data = pd.read_excel(final_output_file_path)

# Display the first few rows to ensure the data is correctly combined
print("Final Combined Data:")
print(final_data.head())

Final combined data saved to /content/Combined_Fareshare_With_Waitrose_Data.xlsx
Final Combined Data:
        Date          Category  \
0 2024-05-01  Fresh vegetables   
1 2024-05-01  Fresh vegetables   
2 2024-05-01  Fresh vegetables   
3 2024-05-01  Fresh vegetables   
4 2024-05-01  Fresh vegetables   

                        Product.brand..name.and.type        Barcode Quantity  \
0             Tesco Mixed Raisins 60G, Fruit and Veg        3424605        8   
1  Tesco Sour Cream Chive Flavoured Pulse & Nut M...  5059697254093        7   
2        Tesco Mixed Peppers 420G (C), Fruit and Veg        3274880        1   
3       Tesco Pink Lady Apple Pot 80G, Fruit and Veg  5057753494520        2   
4  Tesco Nut Bites Mix Baklava Flavour 25G, Fruit...  5059697684951       10   

  Unit Unit.weight Total.weight Unit.price Total.price SheetName  
0   Kg        0.06         0.51       1.15         9.2    Hadley  
1   Kg        0.03         0.18       0.75        5.95    Hadley  
2   Kg     

In [ ]:
# Load the final merged dataset
file_path = "/content/Combined_Fareshare_With_Waitrose_Data.xlsx"  # Update this with your actual file path
final_data = pd.read_excel(file_path)

# Convert dates to only include day, month, and year
final_data['Date'] = pd.to_datetime(final_data['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Update the 'Unit' column: change "Kg" to "Item", leave "Crate" as is
final_data['Unit'] = final_data['Unit'].astype(str).apply(lambda x: 'Item' if x.lower() == 'kg' else x)

# Update headers for weight-related columns to include "(Kg)"
final_data.rename(columns={
    'Unit.weight': 'Unit.weight (Kg)',
    'Total.weight': 'Total.weight (Kg)',
}, inplace=True)

# Save the updated dataset to a new Excel file
output_file_path = "/content/Final_Cleaned_Fareshare_Data.xlsx"
final_data.to_excel(output_file_path, index=False)

print(f"Updated dataset saved to {output_file_path}")

# Display the first few rows of the updated dataframe to verify changes
print(final_data.head())

Updated dataset saved to /content/Final_Cleaned_Fareshare_Data.xlsx
         Date          Category  \
0  2024-05-01  Fresh vegetables   
1  2024-05-01  Fresh vegetables   
2  2024-05-01  Fresh vegetables   
3  2024-05-01  Fresh vegetables   
4  2024-05-01  Fresh vegetables   

                        Product.brand..name.and.type        Barcode Quantity  \
0             Tesco Mixed Raisins 60G, Fruit and Veg        3424605        8   
1  Tesco Sour Cream Chive Flavoured Pulse & Nut M...  5059697254093        7   
2        Tesco Mixed Peppers 420G (C), Fruit and Veg        3274880        1   
3       Tesco Pink Lady Apple Pot 80G, Fruit and Veg  5057753494520        2   
4  Tesco Nut Bites Mix Baklava Flavour 25G, Fruit...  5059697684951       10   

   Unit Unit.weight (Kg) Total.weight (Kg) Unit.price Total.price SheetName  
0  Item             0.06              0.51       1.15         9.2    Hadley  
1  Item             0.03              0.18       0.75        5.95    Hadley  
2  Ite

In [ ]:
# Made some edits in the actiual excel file for simplicity

In [ ]:
final = "/content/James_Fareshare_Data/data/Final_Cleaned_Fareshare_Data.xlsx"


# Load the existing cleaned file
final_data = pd.read_excel(final)

# Define a function to remove anything in parentheses
def remove_parentheses(text):
    return re.sub(r'\s*\(.*?\)\s*', '', str(text))

# Apply the function to the "Location/Store" column
final_data['Location/Store'] = final_data['Location/Store'].apply(remove_parentheses)

# Ensure the Date column is in the correct format (day-month-year)
final_data['Date'] = pd.to_datetime(final_data['Date'], errors='coerce').dt.strftime('%d-%m-%Y')

# Define the new save path
save_path = "/content/Fareshare_Data.xlsx"

# Save the updated dataframe back to the Excel file
final_data.to_excel(save_path, index=False)

# Print a confirmation message
print(f"saved to {save_path}")

saved to /content/Fareshare_Data.xlsx
